<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
# paramter cell do not remove!!
nb_parm=''
question = ''

In [2]:
# Parameters
nb_parm = "bridnok"
question = "describe the characteristic of the brid from the follow context"


In [3]:
import sys
import os

sys.path.append("/home/jovyan/notebooks")
from Framework.module import Utility

## Do the task After this

In [4]:
import time

time.sleep(10)

In [5]:
"""
Since Ollama and Jupyter Notebook run in separate containers, 
Jupyter needs to communicate with Ollama via HTTP requests, 
because they are not in the same process or environment.
"""

'\nSince Ollama and Jupyter Notebook run in separate containers, \nJupyter needs to communicate with Ollama via HTTP requests, \nbecause they are not in the same process or environment.\n'

In [6]:
from langchain_ollama.llms import OllamaLLM

# Explicitly define Ollama's API base URL
llm = OllamaLLM(
    model="deepseek-r1:7b",
    temperature=0,
    # base_url="http://ollama:11434"
    base_url="http://host.docker.internal:11434" # need to refer because it run as separate container the langchain not know what the end point of model are so this is need to be attached

)

# Define a simple question prompt
question = "What is the capital of thailand?"
response = llm.invoke(question)

# Print the response
print(response)


<think>

</think>

The capital of Thailand is Bangkok.


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
# Delete the collection
client.collections.delete("Question")


NameError: name 'client' is not defined

In [ ]:
import weaviate
from weaviate.classes.config import Configure

client = weaviate.connect_to_custom(
    http_host="host.docker.internal",
    http_port=8081,
    http_secure=False,  # Set to True if using HTTPS
    grpc_host="host.docker.internal",
    grpc_port=50051,
    grpc_secure=False,  # Set to True if using a secure gRPC connection
)

print(client.is_ready())  # Should return True if the connection is successful

questions = client.collections.create(
    name="Question",
    vectorizer_config = Configure.Vectorizer.text2vec_ollama(
    api_endpoint="http://host.docker.internal:11434",  # Ensures Weaviate inside Docker can talk to Ollama on host
    model="mxbai-embed-large"                          # The Ollama model name for embedding nomic-embed-text, mxbai-embed-large
    ),
    generative_config=Configure.Generative.ollama(              # Configure the Ollama generative integration
        api_endpoint="http://host.docker.internal:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
        model="deepseek-r1:7b",                                       # The model to use
    )
)

client.close()  # Free up resources

In [ ]:
import weaviate
import requests, json

client = weaviate.connect_to_custom(
    http_host="host.docker.internal",
    http_port=8081,
    http_secure=False,  # Set to True if using HTTPS
    grpc_host="host.docker.internal",
    grpc_port=50051,
    grpc_secure=False,  # Set to True if using a secure gRPC connection
)
resp = requests.get(
    "https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json"
)
data = json.loads(resp.text)

questions = client.collections.get("Question")

with questions.batch.dynamic() as batch:
    for d in data:
        batch.add_object({
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        })
        if batch.number_errors > 10:
            print("Batch import stopped due to excessive errors.")
            break

failed_objects = questions.batch.failed_objects
if failed_objects:
    print(f"Number of failed imports: {len(failed_objects)}")
    print(f"First failed object: {failed_objects[0]}")

client.close()  # Free up resources

In [ ]:
import weaviate
import json

client = weaviate.connect_to_custom(
    http_host="host.docker.internal",
    http_port=8081,
    http_secure=False,  # Set to True if using HTTPS
    grpc_host="host.docker.internal",
    grpc_port=50051,
    grpc_secure=False,  # Set to True if using a secure gRPC connection
)

questions = client.collections.get("Question")

response = questions.query.near_text(
    query="biology",
    limit=2
)

for obj in response.objects:
    print(json.dumps(obj.properties, indent=2))

client.close()  # Free up resources

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OllamaEmbeddings(
                                                model="mxbai-embed-large",  # Or another embedding model, like `nomic-embed-text`
                                                base_url="http://host.docker.internal:11434"
                                            )
                                   )

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = llm = OllamaLLM(
    model="deepseek-r1:7b",
    temperature=0,
    base_url="http://host.docker.internal:11434" 
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")